In [1]:
import duckdb


In [2]:

q = """CREATE TABLE AllergyIntolerance AS SELECT * FROM  read_parquet('AllergyIntolerance.parquet/part-00000-22e1f3fb-af3a-4f8c-bf3d-b2e836512b5d-c000.snappy.parquet')"""
duckdb.sql(q)


In [3]:
base_path = %pwd
print(base_path)

/Users/ilyabeda/projects/sql-on-fhir-py


In [4]:
q = """select * from AllergyIntolerance"""
duckdb.sql(q)

┌───────────────┬──────────────────────┬──────────────────────┬───┬─────────┬──────────────────────┐
│   category    │    clinicalStatus    │         code         │ … │  type   │  verificationStatus  │
│   varchar[]   │ struct(coding stru…  │ struct(coding stru…  │   │ varchar │ struct(coding stru…  │
├───────────────┼──────────────────────┼──────────────────────┼───┼─────────┼──────────────────────┤
│ [environment] │ {'coding': [{'code…  │ {'coding': [{'code…  │ … │ allergy │ {'coding': [{'code…  │
│ [environment] │ {'coding': [{'code…  │ {'coding': [{'code…  │ … │ allergy │ {'coding': [{'code…  │
│ [environment] │ {'coding': [{'code…  │ {'coding': [{'code…  │ … │ allergy │ {'coding': [{'code…  │
│ [environment] │ {'coding': [{'code…  │ {'coding': [{'code…  │ … │ allergy │ {'coding': [{'code…  │
│ [environment] │ {'coding': [{'code…  │ {'coding': [{'code…  │ … │ allergy │ {'coding': [{'code…  │
│ [medication]  │ {'coding': [{'code…  │ {'coding': [{'code…  │ … │ allergy │ {'coding': [{

In [5]:
q = """CREATE TABLE patient AS SELECT * FROM  read_parquet('Patient.parquet/part-00000-d58f2ef8-cc43-48e1-9723-05966f96ca3f-c000.snappy.parquet')"""
duckdb.sql(q)

In [6]:
q = """
SELECT patient.id AS patient_id, patient.name AS patient_name, patient."birthDate" AS "patient_birthDate", patient.gender AS patient_gender 
FROM patient
"""

duckdb.sql(q).show()

┌──────────────────────┬──────────────────────────────────────────────────────────┬───────────────────┬────────────────┐
│      patient_id      │                       patient_name                       │ patient_birthDate │ patient_gender │
│       varchar        │ struct("family" varchar, given varchar[], prefix varch…  │      varchar      │    varchar     │
├──────────────────────┼──────────────────────────────────────────────────────────┼───────────────────┼────────────────┤
│ b0e2bc9c-11c2-3ccf…  │ [{'family': Muller251, 'given': [Rueben647, Antwan357]…  │ 1989-12-02        │ male           │
│ 75369cf4-ab69-a690…  │ [{'family': Champlin946, 'given': [Donny470, Gregorio3…  │ 2008-10-08        │ male           │
│ 513822be-18f9-c962…  │ [{'family': Stehr398, 'given': [Dannielle300, Lu473], …  │ 1971-05-30        │ female         │
│ c0990319-e653-935e…  │ [{'family': Ebert178, 'given': [Veronika907, Josette39…  │ 1960-09-01        │ female         │
│ 913d8550-a4fc-da48…  │ [{'fami

In [7]:
from sqlalchemy import create_engine
eng = create_engine("duckdb:///:memory:")

In [8]:
from sqlalchemy import Column, Integer, Sequence, String, create_engine, JSON, ARRAY
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm.session import Session


In [9]:
Base = declarative_base()


class Patient(Base):
    __tablename__ = "patient"
    
    id = Column(String, primary_key=True) 
    name = Column(ARRAY(JSON))
    birthDate = Column(String) 
    gender = Column(String) 


In [10]:
from sqlalchemy.sql import text
with eng.connect() as con:
    con.execute(text("""CREATE TABLE patient AS SELECT * FROM  read_parquet('Patient.parquet/part-00000-d58f2ef8-cc43-48e1-9723-05966f96ca3f-c000.snappy.parquet')"""))
    con.commit()

In [15]:
with eng.connect() as con:
    for line in con.execute(text("""
select
 name[1].family as family,
 datediff('year', birthDate::date, current_date ) as age,
 gender as gender
from patient
limit 10
""")):
        print(line)

('Muller251', 34, 'male')
('Champlin946', 15, 'male')
('Stehr398', 52, 'female')
('Ebert178', 63, 'female')
('Carter549', 32, 'female')
('Kuvalis369', 45, 'male')
('Jacobs452', 60, 'female')
('Kihn564', 57, 'female')
('Dickens475', 12, 'female')
('Abshire638', 38, 'male')


In [53]:
from sqlalchemy import select, func

session = Session(bind=eng)

q = select(Patient.name[1]['family'])
#session.execute(q).all()
print(q)

SELECT (patient.name[:name_1])[:param_1] AS anon_1 
FROM patient
